# Scan LQDR MasterChef instead of Gauge

In [47]:
import os
import json
import pandas as pd
from dotenv import load_dotenv

load_dotenv(os.path.expanduser('~/Work/morpheus-swap/.env'))

True

In [48]:
with open(os.path.expanduser('~/Work/chaino/var/lqdr-pills-master-chef-balance.json'), 'r') as f:
    userinfo = json.load(f)

results = {}
for task_id in userinfo.keys():
    results.update(userinfo[task_id])

df = pd.DataFrame.from_dict(results, orient='index', columns=['amount', 'reward_debt'])
df['address'] = df.index
df['amount'] = df['amount'] / (10**18)
df = df.reset_index(drop=True)
del df['reward_debt']

df.head()

,amount,address
0,766.585645,0xa4722a7A8a79E0de7B568559261D789f0B80baDC
1,0.0,0x326A7D91B6F767458599E7d93DFc850d2e083d30
2,0.0,0x4BC4Da7839a3c987D44F0DBDAcBDd1f53243Ba24
3,0.0,0xE1e96F427297C1A7c597cAEDF127D7906F1f2794
4,0.0,0x3B97fD8F6fbF336672a80a9C3072b60c1AC37914


In [49]:
# the total volume of LP-esque tokens minted by LQDR to represent staked Spirit LP
lqdr_spirit_lp_total = df['amount'].sum()
lqdr_spirit_lp_total

1154249.6829276592

In [50]:
filename = os.path.join(os.environ['DATA_PATH'], '2022-04-20-pills-contracts.csv.gz')
df_contracts = pd.read_csv(filename)
df_contracts.head()

,address,pills_erc20_balance,beets_lp_erc20_balance,neo_pool_0_amount,neo_pool_0_reward_debt,neo_pool_1_amount,neo_pool_1_reward_debt,spirit_amount,spirit_reward_debt
0,0x9C775D3D66167685B2A3F4567B548567D2875350,2.415931e+06,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
1,0xF04588329c3104779796F279A9df3803Ae83dB7B,0.000000e+00,0.0,50659.894950,1463.820474,0.0,0.0,0.0,0.0
2,0x326A7D91B6F767458599E7d93DFc850d2e083d30,2.882333e+06,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
3,0x2728e32174FC95De00695007c9134fA03dC8b434,0.000000e+00,0.0,53376.318198,1611.750893,0.0,0.0,0.0,0.0
4,0xD488F4585Ab28D003324253be5D791aC103c0b57,8.730000e-16,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0


In [51]:
# the total amount of PILLS controlled by the Spirit LP, according to PILLS ERC-20
spirit_pills_volume = df_contracts.loc[df_contracts['address'] == '0x9C775D3D66167685B2A3F4567B548567D2875350']['pills_erc20_balance']
spirit_pills_volume = float(spirit_pills_volume)
spirit_pills_volume

2415931.0318463827

In [52]:
# per-address volume of PILLS as proportion of Spirit LP staked with LQDR
df['lqdr_pills_balance'] = spirit_pills_volume * (df['amount'] / lqdr_spirit_lp_total)
df.head()

,amount,address,lqdr_pills_balance
0,766.585645,0xa4722a7A8a79E0de7B568559261D789f0B80baDC,1604.521167
1,0.0,0x326A7D91B6F767458599E7d93DFc850d2e083d30,0.0
2,0.0,0x4BC4Da7839a3c987D44F0DBDAcBDd1f53243Ba24,0.0
3,0.0,0xE1e96F427297C1A7c597cAEDF127D7906F1f2794,0.0
4,0.0,0x3B97fD8F6fbF336672a80a9C3072b60c1AC37914,0.0


In [53]:
del df['amount']
filename = os.path.join(os.environ['DATA_PATH'], 'lqdr-pills-master-chef-balance.csv')
df.to_csv(filename)

df['lqdr_pills_balance'].sum()

2415931.031846382